<a href="https://colab.research.google.com/github/vrathi101/SIGCOMMInclusivity/blob/main/Webscraping_Cleaning_SIGCOMMData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and import necessary Python libraries for subsequent code (these libraries will be used throughout the Google Colabs.
1. Fuzzywuzzy is used for string matching (comparing strings which aren't identical but may have differences caused by typos, abbreviations, etc.). I used this when there weren't exact matches between the committee member dataframe and the researchers csv.
2. Levenshtein is used to measure the "distance" between strings which will be used for fuzzy comparing.
3. Beautifulsoup4 is used for webscraping. I used webscraping to extract the committee members for each year for SIGCOMM.
4. Gender-guesser is used to guess the gender of a person based on their first name. It have 6 possible outputs: 'male', 'mostly_male', 'female', 'mostly_female', 'andy', and 'unknown'. I used this for the names which had a fuzzy ratio of less than the fixed threshold when compared to every name in the researchers csv.
5. Unidecode is used to turn weird characters, like letters with accents or unprintable characters, into regular ones to make it easier for matching.
6. GeonamesCache was used to convert country names into the continents where they are located.
7. GeoText is used to extract words like a city or location from a longer text.
8. Ipywidgets adds interactivity to dataframes where users can visualize and organize the data as they wish. The library uses click events to show the desired results.

In [ ]:
#install all packages
!pip install fuzzywuzzy
!pip install levenshtein
!pip install requests beautifulsoup4
!pip install gender-guesser
!pip install unidecode
!pip install GeonamesCache
!pip install GeoText
!pip install ipywidgets
!pip install ipywidgets

Import all of the libraries we will be using.

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import gender_guesser.detector as gender_detector
from unidecode import unidecode
import re
from geonamescache import GeonamesCache
from geotext import GeoText

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Using webscraping to extract all the organizational committee and program committee members and their institutions for SIGCOMM from 2014-2023, inclusive, and then adding all the names/regions, categorized by year and committee type, into a dataframe.

In [4]:
# Create an empty DataFrame with columns for name, year, and committee
dfSigcomm = pd.DataFrame(columns=['name', 'year', 'committee','region'])
committee="Organizing"
# Loop through the years 2009-2023
for year in range(2009, 2024):
  # Determine the URL based on the year
  if year >= 2012:
    if year in [2016, 2014, 2013, 2012]:
      url = f"https://conferences.sigcomm.org/sigcomm/{year}/organization.php"
    elif year == 2015:
      url = "https://conferences.sigcomm.org/sigcomm/2015/organisation.php"
    else:
      url = f"https://conferences.sigcomm.org/sigcomm/{year}/org-committee.html"
    # Send a GET request to the URL and retrieve the response
    response = requests.get(url)
    # Create a BeautifulSoup object to parse the HTML response
    soup = BeautifulSoup(response.text, "html.parser")
    # Select the HTML elements that contain the names of committee members
    name_elements = soup.select('div.ui-grid-a div.ui-block-a p')
    # Extract the names from the HTML elements and strip any whitespace
    member_name = [name.text.strip() for name in name_elements]
    # Select the HTML elements that contain the regions of committee members
    region_elements = soup.select('div.ui-grid-a div.ui-block-b p')
    # Extract the regions from the HTML elements and strip any whitespace
    region_name = [region.text.strip() for region in region_elements]
    #add data to a dataframe and concatenate with the original
    df = pd.DataFrame({'name': member_name, 'year': year, 'committee': committee, 'region': region_name})
    dfSigcomm = pd.concat([dfSigcomm, df], ignore_index=False)
  elif year in [2011, 2010, 2009]:
      # Create the URL with the specified year
      url = f"https://conferences.sigcomm.org/sigcomm/{year}/organization.php"
      # Send a GET request to the URL and retrieve the response
      response = requests.get(url)
      # Create a BeautifulSoup object to parse the HTML response
      soup = BeautifulSoup(response.text, "html.parser")
      # Find the <div> element with the id "contents"
      contents_div = soup.find("div", id="contents")
      if contents_div:
          if year == 2010:
              # Find the <h2> element with the text "Organizing Committee"
              committee_header = contents_div.find("h2", string="Organizing Committee")
          else:
              # Find the <h2> element with the text "Committee"
              committee_header = contents_div.find("h2", string="Committee")
          if committee_header:
              # Find the table following the <h2> element
              committee_table = committee_header.find_next_sibling("table")
              if committee_table:
                  # Find all <tr> elements within the table's <tbody>
                  committee_members = committee_table.find_all("tr")
                  for member in committee_members:
                      # Find the first <td> element within each <tr>
                      name_cell = member.find_all("td")
                      if name_cell:
                          member_name = name_cell[0].get_text(strip=True)
                          if len(name_cell) >= 2:
                              region = name_cell[1]
                              region_name = region.get_text(strip=True)
                              # Add the name to the final DataFrame
                              df = pd.DataFrame({'name': [member_name], 'year': year, 'committee': 'Organizing', 'region': [region_name]})
                              dfSigcomm = pd.concat([dfSigcomm, df], ignore_index=True)
                          else:
                              df = pd.DataFrame({'name': [member_name], 'year': year, 'committee': 'Organizing'})
                              dfSigcomm = pd.concat([dfSigcomm, df], ignore_index=True)
# Print the final DataFrame
print(dfSigcomm.head(10))
print(len(dfSigcomm))
print(dfSigcomm.groupby(['year', 'committee'])['year', 'committee'].count())

                        name  year   committee  \
0            Pablo Rodriguez  2009  Organizing   
1             Ernst Biersack  2009  Organizing   
2  Konstantina  Papagiannaki  2009  Organizing   
3                Luigi Rizzo  2009  Organizing   
4            Christophe Diot  2009  Organizing   
5                Dolors Sala  2009  Organizing   
6      Jaudelice de Oliveira  2009  Organizing   
7  Balachander Krishnamurthy  2009  Organizing   
8               Ant Rowstron  2009  Organizing   
9              Laurent Mathy  2009  Organizing   

                            region  
0       Telefonica Research, Spain  
1                  Eurecom, France  
2       Intel Labs Pittsburgh, USA  
3        Universit� di Pisa, Italy  
4                  Thomson, France  
5  Universitat Pompeu Fabra, Spain  
6           Drexel University, USA  
7          AT&T Labs-Research, USA  
8           Microsoft Research, UK  
9         Lancaster University, UK  
427
                 year  committee
year 

<ipython-input-4-4f7bc7a76dac>:68: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  print(dfSigcomm.groupby(['year', 'committee'])['year', 'committee'].count())


In [5]:
# Loop through the years 2009-2023
for year in range(2009, 2024):
  if year >= 2012:
    # Determine the URL based on the year
    if year in [2016, 2015, 2014]:
      url = f"https://conferences.sigcomm.org/sigcomm/{year}/pc.php"
    elif year in [2013, 2012]:
      url = f"https://conferences.sigcomm.org/sigcomm/{year}/pclist.php"
    else:
      url = f"https://conferences.sigcomm.org/sigcomm/{year}/tpc.html"
    # Send a GET request to the URL and retrieve the response
    response = requests.get(url)
    # Create a BeautifulSoup object to parse the HTML response
    soup = BeautifulSoup(response.text, "html.parser")
    # Select the HTML elements that contain the names of committee members
    name_elements = soup.select('div.ui-grid-a div.ui-block-a p')
    # Extract the names from the HTML elements and strip any whitespace
    member_name = [name.text.strip() for name in name_elements]
    # Select the HTML elements that contain the regions of committee members
    region_elements = soup.select('div.ui-grid-a div.ui-block-b p')
    # Extract the regions from the HTML elements and strip any whitespace
    region_name = [region.text.strip() for region in region_elements]
    #add data to a dataframe and concatenate with the original
    df = pd.DataFrame({'name': member_name, 'year': year, 'committee': 'Program', 'region': region_name})
    dfSigcomm = pd.concat([dfSigcomm, df], ignore_index=False)
  else:
    # Create the URL with the specified year
    url = f"https://conferences.sigcomm.org/sigcomm/{year}/organization.php"
    # Send a GET request to the URL and retrieve the response
    response = requests.get(url)
    # Create a BeautifulSoup object to parse the HTML response
    soup = BeautifulSoup(response.text, "html.parser")
    # Find the <div> element with the id "contents"
    contents_div = soup.find("div", id="contents")
    if contents_div:
        if year == 2010:
            # Find the <h2> element with the text "Organizing Committee"
            committee_header = contents_div.find("h2", string="Technical Program Committee")
        else:
            # Find the <h2> element with the text "Committee"
            committee_header = contents_div.find("h2", string="Program Committee")
        if committee_header:
            # Find the table following the <h2> element
            committee_table = committee_header.find_next_sibling("table")
            if committee_table:
                # Find all <tr> elements within the table's <tbody>
                committee_members = committee_table.find_all("tr")
                for member in committee_members:
                      # Find the first <td> element within each <tr> which will have the name
                      name_cell = member.find_all("td")
                      if name_cell:
                          #search for the other <td> element which will have the region
                          member_name = name_cell[0].get_text(strip=True)
                          if len(name_cell) >= 2:
                              region = name_cell[1]
                              region_name = region.get_text(strip=True)
                              # Add the name to the final DataFrame via concatenation
                              df = pd.DataFrame({'name': [member_name], 'year': year, 'committee': 'Program', 'region': [region_name]})
                              dfSigcomm = pd.concat([dfSigcomm, df], ignore_index=True)
                          else:
                              df = pd.DataFrame({'name': [member_name], 'year': year, 'committee': 'Program'})
                              dfSigcomm = pd.concat([dfSigcomm, df], ignore_index=True)

# Print the final DataFrame
print(dfSigcomm.head(5))
print(len(dfSigcomm))
print(dfSigcomm.groupby(['year', 'committee'])['year', 'committee'].count())

                        name  year   committee                      region
0            Pablo Rodriguez  2009  Organizing  Telefonica Research, Spain
1             Ernst Biersack  2009  Organizing             Eurecom, France
2  Konstantina  Papagiannaki  2009  Organizing  Intel Labs Pittsburgh, USA
3                Luigi Rizzo  2009  Organizing   Universit� di Pisa, Italy
4            Christophe Diot  2009  Organizing             Thomson, France
1250
                 year  committee
year committee                  
2009 Organizing    21         21
     Program       60         60
2010 Organizing    24         24
     Program       50         50
2011 Organizing    17         17
     Program       52         52
2012 Organizing    18         18
     Program       51         51
2013 Organizing    25         25
     Program       35         35
2014 Organizing    26         26
     Program       54         54
2015 Organizing    33         33
     Program       51         51
2016 Organizing  

<ipython-input-5-9d06825e4c55>:67: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  print(dfSigcomm.groupby(['year', 'committee'])['year', 'committee'].count())


Making adjustments to the dataframe to proceed with further analysis.

In [6]:
#removing any duplicate names that appear in the same year and committee type
grouped = dfSigcomm.groupby(['year', 'committee'])
dfSigcomm = dfSigcomm[~grouped['name'].transform(lambda x: x.duplicated())]
#reindexing the dataframe to avoid future problems
dfSigcomm.reset_index(drop=True, inplace=True)
#printing
print(dfSigcomm.head(5))
print(len(dfSigcomm))

                        name  year   committee                      region
0            Pablo Rodriguez  2009  Organizing  Telefonica Research, Spain
1             Ernst Biersack  2009  Organizing             Eurecom, France
2  Konstantina  Papagiannaki  2009  Organizing  Intel Labs Pittsburgh, USA
3                Luigi Rizzo  2009  Organizing   Universit� di Pisa, Italy
4            Christophe Diot  2009  Organizing             Thomson, France
1241


Handling the case where names where unprintable characters are displayed for names or regions. We replace these with the actual characters using Unidecode and dictionary mappings.

In [7]:
# Function to decode the string with fallback encoding
def decode_with_fallback(text, primary_encoding, fallback_encoding):
    try:
        if isinstance(text, str):
            return text.encode(primary_encoding).decode(primary_encoding)
        else:
            return str(text).encode(fallback_encoding).decode(fallback_encoding)
    except UnicodeDecodeError:
        return str(text).encode(fallback_encoding).decode(fallback_encoding)

# Assign existing dataframe to a new one
dfOutput = dfSigcomm
# Apply the decoding function to create the 'cleanedName' and 'cleanedRegion' columns
dfOutput['cleanedName'] = dfOutput['name'].apply(lambda x: decode_with_fallback(x, 'utf-8', 'unicode_escape'))
dfOutput['cleanedRegion'] = dfOutput['region'].apply(lambda x: decode_with_fallback(x, 'utf-8', 'unicode_escape'))
dfOutput['cleanedName'] = dfOutput['cleanedName'].str.replace('�', '')
dfOutput['cleanedRegion'] = dfOutput['cleanedRegion'].str.replace('�', '')
# Print the updated DataFrame
print(dfOutput.head(10))

                        name  year   committee  \
0            Pablo Rodriguez  2009  Organizing   
1             Ernst Biersack  2009  Organizing   
2  Konstantina  Papagiannaki  2009  Organizing   
3                Luigi Rizzo  2009  Organizing   
4            Christophe Diot  2009  Organizing   
5                Dolors Sala  2009  Organizing   
6      Jaudelice de Oliveira  2009  Organizing   
7  Balachander Krishnamurthy  2009  Organizing   
8               Ant Rowstron  2009  Organizing   
9              Laurent Mathy  2009  Organizing   

                            region                cleanedName  \
0       Telefonica Research, Spain            Pablo Rodriguez   
1                  Eurecom, France             Ernst Biersack   
2       Intel Labs Pittsburgh, USA  Konstantina  Papagiannaki   
3        Universit� di Pisa, Italy                Luigi Rizzo   
4                  Thomson, France            Christophe Diot   
5  Universitat Pompeu Fabra, Spain                Dolors Sa

<ipython-input-7-3e5ca6d39358>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOutput['cleanedName'] = dfOutput['name'].apply(lambda x: decode_with_fallback(x, 'utf-8', 'unicode_escape'))
<ipython-input-7-3e5ca6d39358>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOutput['cleanedRegion'] = dfOutput['region'].apply(lambda x: decode_with_fallback(x, 'utf-8', 'unicode_escape'))
<ipython-input-7-3e5ca6d39358>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [8]:
nameDict = {'Ãtalo Cunha':'Italo Cunha',
'AndrÃƒÂ¡s CsÃƒÂ¡szÃƒÂ¡r':'Andras Csaszar',
'AndrÃ¡s CsÃ¡szÃ¡r':'Andras Csaszar',
'BalÃƒÂ¡zs Sonkoly':'Balazs Sonkoly',
'BalÃ¡zs Sonkoly':'Balazs Sonkoly',
'Emin GÃ¼n Sirer':'Emin Gun Sirer',
'FabÃ­ola Greve':'Fabiola Greve',
'FabiÃ¡n Bustamante':'Fabian Bustamente',
'FabiÃ¡n E. Bustamante':'Fabian E. Bustamente',
'FabiÃƒÂ¡n Bustamante':'Fabian Bustamente',
'GÃƒÂ¡bor RÃƒÂ©tvÃƒÂ¡ri':'Gabor Retvari',
'GÃ¡bor RÃ©tvÃ¡ri': 'Gabor Retvari',
'IstvÃƒÂ¡n GÃƒÂ³dor':'Istvan Godor',
'IstvÃ¡n GÃ³dor':'Istvan Godor',
'JÃ¶rg Liebeherr':'Jaorg Liebeherr',
'JÃ¶rg Ott':'Jaorg Ott',
'JÃƒÂ¡nos Tapolcai':'Janos Tapolcai',
'JÃ¡nos Tapolcai':'Janos Tapolcai',
'Matthias WÃƒÂ¤hlisch':'Matthias Waschlisch',
'Matthias WÃ¤hlisch':'Matthias Waschlisch',
'Robert SoulÃƒÂ©':'Robert Soulac',
'Robert SoulÃ©': 'Robert Soulac',
'RubÃƒÂ©n Cuevas':'Rubaon Cuevas',
'RubÃ©n Cuevas':'Rubaon Cuevas',
'ZalÃƒÂ¡n Heszberger':'Zalan Heszberger',
'ZalÃ¡n Heszberger':'Zalan Heszberger',
'ZoltÃƒÂ¡n LÃƒÂ¡zÃƒÂ¡r':'Zoltan Lazar',
'ZoltÃ¡n LÃ¡zÃ¡r':'Zoltan Lazar',
'Cecilia Testart/p>': 'Cecilia Testart'}
dfOutput['cleanedName'] = dfOutput['cleanedName'].map(nameDict).fillna(dfOutput['cleanedName'])
print(dfOutput['cleanedName'])

0                 Pablo Rodriguez
1                  Ernst Biersack
2       Konstantina  Papagiannaki
3                     Luigi Rizzo
4                 Christophe Diot
                  ...            
1236                   Yiting Xia
1237                 Yiying Zhang
1238                       Yu Hua
1239                   Yunxin Liu
1240                Zhizhen Zhong
Name: cleanedName, Length: 1241, dtype: object


<ipython-input-8-326921f1e225>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOutput['cleanedName'] = dfOutput['cleanedName'].map(nameDict).fillna(dfOutput['cleanedName'])


In [9]:
regionDict = {'Universitï¿½ di Pisa, Italy':'Universita di Pisa',
'Universitat Politï¿½cnica de Catalunya, Spain':'Universitat Politecnica de Catalunya',
'Universidad Autï¿½noma de Madrid, Spain':'Universidad Autonoma de Madrid',
'Freie UniversitÃƒÂ¤t Berlin, Germany':'Freie Universitat Berlin',
'Freie UniversitÃ¤t Berlin, Germany':'Freie Universitat Berlin',
'University of WisconsinÃ¢Â€Â“Madison':'University of Wisconsin-Madison',
'University of WisconsinâMadison':'University of Wisconsin-Madison',
'Technische UniversitÃƒÂ¤t MÃƒÂ¼nchen':'Technische Universitat Munchen',
'Technische UniversitÃ¤t MÃ¼nchen':'Technische Universitat Munchen',
'UniversitÃ  di Pisa, Italy, USA':'Universita di Pisa',
'ETH ZÃƒÂ¼rich':'ETH Zurich',
'ETH ZÃ¼rich':'ETH Zurich',
'UniversitÃƒÂ  della Svizzera italiana':'Universita della Svizzera Italiana',
'UniversitÃƒÂ© catholique de Louvain':'Universite Catholique de Louvain',
'''UniversitÃ© catholique de
              Louvain''':'Universite Catholique de Louvain',
'University of Illinois UrbanaÃ¢Â€Â“Champaign':'University of Illinois Urbana-Champaign',
'University of Illinois UrbanaâChampaign':'University of Illinois Urbana-Champaign',
'AT&T Labs - Research, USANikolaos LaoutarisTelefonica Research, SpainRatul MahajanMicrosoft Research, USADavid OranCisco, USAJitendra PadhyeMicrosoft Research, USAVenkat PadmanabhanMicrosoft Research, IndiaAdrian PerrigCarnegie Mellon University, USALili QiuUniversity of Texas at Austin, USAByrav RamamurthyUniversity of Nebraska-Lincoln, USADanny RazTechnion, IsraelScott RixnerRice University, USATimothy RoscoeETH Zurich, SwitzerlandMema RoussopoulosUniversity of Athens, GreeceStefan SavageU.C. San Diego, USASrinivasan SeshanCarnegie Mellon University, USADevavrat ShahMIT, USAScott ShenkerU.C. Berkeley, USAEmin Gün SirerCornell University, USAVijay SivaramanUniversity of New South Wales, AustraliaKun TanMicrosoft Research, ChinaAmin VahdatGoogle / U.C. San Diego, USAHelen WangMicrosoft Research, USAGordon WilfongBell Labs Research, USAWalter WillingerAT&T Labs - Research, USAAlec WolmanMicrosoft Research, USAYinglian XieMicrosoft Research, USARichard YangYale University, USAHaifeng YuNational University of Singapore, SingaporeEllen ZeguraGeorgia Tech, USAYin ZhangUniversity of Texas at Austin, USAYongguang ZhangMicrosoft Research, ChinaHeather ZhengU.C. Santa Barbara, USA':'AT&T Labs'}
dfOutput['cleanedRegion'] = dfOutput['cleanedRegion'].map(nameDict).fillna(dfOutput['cleanedRegion'])
print(dfOutput['cleanedRegion'])

0                              Telefonica Research, Spain
1                                         Eurecom, France
2                              Intel Labs Pittsburgh, USA
3                                Universit di Pisa, Italy
4                                         Thomson, France
                              ...                        
1236                 Max Planck Institute for Informatics
1237                  University of California, San Diego
1238        Huazhong University of Science and Technology
1239    Institute for AI Industry Research (AIR), Tsin...
1240                Massachusetts Institute of Technology
Name: cleanedRegion, Length: 1241, dtype: object


<ipython-input-9-96c230394d59>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOutput['cleanedRegion'] = dfOutput['cleanedRegion'].map(nameDict).fillna(dfOutput['cleanedRegion'])


In [10]:
# Print column names and data types
for column_name, column_type in zip(dfOutput.columns, dfOutput.dtypes):
    print(f"Column Name: {column_name}, Data Type: {column_type}")
print(len(dfOutput))
print(len(dfSigcomm))
print(dfOutput.head(5))
print()
print(dfOutput['name'].head(5))

Column Name: name, Data Type: object
Column Name: year, Data Type: object
Column Name: committee, Data Type: object
Column Name: region, Data Type: object
Column Name: cleanedName, Data Type: object
Column Name: cleanedRegion, Data Type: object
1241
1241
                        name  year   committee                      region  \
0            Pablo Rodriguez  2009  Organizing  Telefonica Research, Spain   
1             Ernst Biersack  2009  Organizing             Eurecom, France   
2  Konstantina  Papagiannaki  2009  Organizing  Intel Labs Pittsburgh, USA   
3                Luigi Rizzo  2009  Organizing   Universit� di Pisa, Italy   
4            Christophe Diot  2009  Organizing             Thomson, France   

                 cleanedName               cleanedRegion  
0            Pablo Rodriguez  Telefonica Research, Spain  
1             Ernst Biersack             Eurecom, France  
2  Konstantina  Papagiannaki  Intel Labs Pittsburgh, USA  
3                Luigi Rizzo    Universi

Here we use the geography libraries: If the institution read from the SIGCOMM website has a country listed, it will add the continent of the country to a dataframe column. If instead the institution is a specific location (like a university) but doesn't have the country listed, we map the location to a country which will be used to determine the continent.

In [11]:
def get_continent(place):
    # Mapping of location abbreviations to country names
    location_mappings = {
        'NUS': 'Singapore',
        'Columbia|US|USA|Harvard|Stanford|George Washington|Irvine|West Point|Connecticut|Princeton|Purdue|Minnesota|Austin|San Diego|Pennsylvania|NYU|Chicago|Virginia|MIT|Georgia|Duke|Northeastern|California|Northwestern|Massachusetts|Carnegie|UCLA|CMU|Stony Brook|Wisconsin|Wisconsin-Madison|Rice|Colorado|Rutgers|Illinois|Boston|Florida|New York|Maryland|Michigan|Yale|Washington|Texas|Carolina|UC|Cornell|Brown|Oregon|USC|UCSB|Amherst|UIUC|Johns Hopkins|UWM|UWisc|Brown|UMich|UMass|Redmond|ICSI': 'United States',
        'UK|London|Cambridge|Oxford|Southampton': 'United Kingdom',
        'Karlstad|KTH': 'Sweden',
        'Hong Kong|Chiao|Huazhong|Tsinghua|Fudan|Peking|HKUST|NTU': 'China',
        'EPFL|Zurich|Svizzera|ETHZ': 'Switzerland',
        'MPI-SWS|Planck|Munich|Brandenburg|Technische|Potsdam|MPI': 'Germany',
        'Delft|Amsterdam|Twente': 'Netherlands',
        'Jerusalem|Reichman': 'Israel',
        'KAIST|Korea': 'South Korea',
        'UNSW|Melbourne|Adelaide': 'Australia',
        'Toronto|Waterloo': 'Canada',
        'Budapest': 'Hungary',
        'Abdullah': 'South Arabia',
        'IMDEA': 'Spain',
        'Linz|AIT|Austrian': 'Austria',
        'Campinas|Passo Fundo': 'Brazil',
        'Lisbon': 'Portugal',
        'IIT': 'India',
        'UCLouvain|Louvain': 'Belgium',
        'Lahore|LUMS': 'Pakistan',
        'Waikato': 'New Zealand',
        'Bucharest': 'Romania',
        'Inria': 'France',
        'IIJ|NII': 'Japan'
    }
    # Wording manipulations to extract mappings
    for abbreviation, full_name in location_mappings.items():
        abbreviation_list = re.split(r'\||\s+', abbreviation)
        abbreviation_regex = r'\b({})\b'.format('|'.join(re.escape(abbr) for abbr in abbreviation_list))
        place = re.sub(abbreviation_regex, full_name, place, flags=re.IGNORECASE)

    # Extract country from the place name
    countries = GeoText(place).countries
    if countries:
        country = countries[0]
    else:
        return ''

    # Remove extra text after country name
    place = place.replace(country, '').strip(', ')

    # Get continent based on country
    geonames = GeonamesCache()
    country_info = geonames.get_countries_by_names().get(country)
    if country_info:
        continent = country_info['continentcode']
        if not continent:
            return ''
    else:
        return ''

    # If additional text exists, extract country from it
    if place:
        additional_countries = GeoText(place).countries
        if additional_countries:
            additional_country = additional_countries[0]
            additional_country_info = geonames.get_countries_by_names().get(additional_country)
            if additional_country_info:
                continent = additional_country_info['continentcode']
                if continent:
                    return continent

    return continent

# Apply the get_continent function to the 'cleanedRegion' column in the DataFrame
dfOutput['Continent'] = dfOutput['cleanedRegion'].apply(get_continent)
dfOutput['Continent'].replace('NA', 'NorAm', inplace=True)
# Print the updated DataFrame
print(dfOutput.head(10))

                        name  year   committee  \
0            Pablo Rodriguez  2009  Organizing   
1             Ernst Biersack  2009  Organizing   
2  Konstantina  Papagiannaki  2009  Organizing   
3                Luigi Rizzo  2009  Organizing   
4            Christophe Diot  2009  Organizing   
5                Dolors Sala  2009  Organizing   
6      Jaudelice de Oliveira  2009  Organizing   
7  Balachander Krishnamurthy  2009  Organizing   
8               Ant Rowstron  2009  Organizing   
9              Laurent Mathy  2009  Organizing   

                            region                cleanedName  \
0       Telefonica Research, Spain            Pablo Rodriguez   
1                  Eurecom, France             Ernst Biersack   
2       Intel Labs Pittsburgh, USA  Konstantina  Papagiannaki   
3        Universit� di Pisa, Italy                Luigi Rizzo   
4                  Thomson, France            Christophe Diot   
5  Universitat Pompeu Fabra, Spain                Dolors Sa

<ipython-input-11-93c951e91489>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOutput['Continent'] = dfOutput['cleanedRegion'].apply(get_continent)
<ipython-input-11-93c951e91489>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOutput['Continent'].replace('NA', 'NorAm', inplace=True)


Filling in values which are still missing or need to be changed.

In [12]:
data_dict = {
    "David Oran": ("Unknown", "NorAm"),
    "Sam Leffler": ("Google", "NorAm"),
    "Balachander Krishnamurthy": ("AT&T Labs", "NorAm")
}
for name, (region, continent) in data_dict.items():
    dfOutput.loc[dfOutput['cleanedName'] == name, 'region'] = region
    dfOutput.loc[dfOutput['cleanedName'] == name, 'cleanedRegion'] = region
    dfOutput.loc[dfOutput['cleanedName'] == name, 'Continent'] = continent

We write the current dataframe into a CSV file which will be read and analyzed in the next Google Colab.

In [13]:
dfOutput.to_csv("/content/gdrive/My Drive/Colab Notebooks/save_data/PublisherNames.csv", sep=',', index=False, encoding='utf-8')